### Notebook to pickle data needed for latency_analysis.py file

import sys
sys.path.append("..")

In [1]:
import src.config

In [2]:
from src.datasets.loading import statcan, ookla

In [3]:
import numpy as np 
import pandas as pd
import geopandas as gp
import pickle

In [4]:
ookla_tiles = ookla.canada_tiles()

In [5]:
da_pops = statcan.dissemination_areas_populations()

/home/jovyan/src/datasets/loading/statcan.py:234: DtypeWarning: Columns (3,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(POP_FILE)


In [6]:
o = gp.read_file(src.config.OVERLAYS_DIR / 'tile_das_overlay') #this can take a few minutes to load.
tile_da_label = o.dropna(subset=['DAUID','quadkey']).sort_values(by=['quadkey','tile_frac'],ascending=False).drop_duplicates(subset='quadkey', keep='first')
tile_da_label['quadkey'] = tile_da_label['quadkey'].astype(int)
tile_da_label['DAUID'] = tile_da_label['DAUID'].astype(int)

In [7]:
speed_data = ookla.speed_data(ookla.available_files().path)

In [8]:
## merge dissemination area (DA) populations with ookla tiles (already combined with other statcan data)
features_table = tile_da_label.merge(da_pops, on='DAUID', how='left')
features_table['DAPOP'] = features_table['DAPOP'].fillna(0).astype(int)
del features_table['GEO_NAME']
features_table = pd.DataFrame(features_table)
del features_table['geometry']

# take all ookla tiles, merge the speeds data and tile labels and populations
features_table = ookla_tiles.merge(speed_data, on='quadkey').merge(features_table, on='quadkey')

In [10]:
features_table

,quadkey,geometry,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,conn_type,year,quarter,...,CMANAME,CMATYPE,CTUID,CTNAME,ADAUID,das_area,tile_area,tile_frac,das_frac,DAPOP
0,302303331012303,"POLYGON ((-73.36121 45.53329, -73.35571 45.533...",130704,26290,11,8,6,fixed,2019,1,...,Montréal,B,4620855.01,0855.01,24580012,1.654880e+05,189024.729026,0.328401,0.375109,400
1,302303331012303,"POLYGON ((-73.36121 45.53329, -73.35571 45.533...",155185,35459,12,23,5,fixed,2019,2,...,Montréal,B,4620855.01,0855.01,24580012,1.654880e+05,189024.729026,0.328401,0.375109,400
2,302303331012303,"POLYGON ((-73.36121 45.53329, -73.35571 45.533...",49799,11468,15,60,12,fixed,2019,3,...,Montréal,B,4620855.01,0855.01,24580012,1.654880e+05,189024.729026,0.328401,0.375109,400
3,302303331012303,"POLYGON ((-73.36121 45.53329, -73.35571 45.533...",55197,14922,14,27,13,fixed,2019,4,...,Montréal,B,4620855.01,0855.01,24580012,1.654880e+05,189024.729026,0.328401,0.375109,400
4,302303331012303,"POLYGON ((-73.36121 45.53329, -73.35571 45.533...",73777,12470,14,20,10,fixed,2020,1,...,Montréal,B,4620855.01,0855.01,24580012,1.654880e+05,189024.729026,0.328401,0.375109,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857549,303301220300233,"POLYGON ((-53.24524 47.37231, -53.23975 47.372...",54364,48952,34,1,1,mobile,2022,3,...,NaN,NaN,NaN,NaN,10010039,3.173819e+07,173889.125962,1.000000,0.005479,640
3857550,303301220300321,"POLYGON ((-53.23425 47.37603, -53.22876 47.376...",56804,48529,35,4,2,mobile,2022,3,...,NaN,NaN,NaN,NaN,10010039,3.173819e+07,173859.261798,1.000000,0.005478,640
3857551,303301221112323,"POLYGON ((-52.79480 47.46152, -52.78931 47.461...",175390,44981,33,32,2,mobile,2022,3,...,St. John's,B,0010017.00,0017.00,10010025,1.622828e+08,173173.794451,0.856408,0.000914,2400
3857552,303301221332020,"POLYGON ((-52.82227 47.30158, -52.81677 47.301...",103433,9563,56,1,1,mobile,2022,3,...,St. John's,B,0010110.00,0110.00,10010031,9.353094e+06,174457.517212,0.579382,0.010807,490


In [17]:
output_dir = src.config.DATA_DIRECTORY / 'processed' 
output_dir.mkdir(exist_ok = True)
all_years_latency = output_dir / 'all_years_latency.pickle'
with open(all_years_latency, 'wb') as f:
    pickle.dump(features_table, f)


In [18]:
avg_df = features_table.groupby(['CMANAME']).apply(lambda x: (x.loc[:, ['avg_lat_ms','avg_d_Mbps', 'avg_u_Mbps']].mean())).reset_index()

KeyError: "['avg_d_Mbps', 'avg_u_Mbps'] not in index"